In [72]:
pth = "/home_local/kkotzen/databases/mesa/polysomnography/sleep/mesa-sleep-6566-nsrr.xml"

plt.close('all')
sleep = sleep_extract_30s_epochs(pth)
sleep[sleep==5]=4
sleep[sleep==2]=1
sleep[sleep==3]=2
sleep[sleep==4]=3
sleep = clean_sleep_2(sleep)
t = np.arange(len(sleep))
t_, s_ = hold_line(t, sleep)

plt.plot(t_,s_)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal, interpolate
import copy
import os 
import pandas as pd
from tqdm import tqdm

from numpy.core._multiarray_umath import ndarray
import xml.etree.ElementTree as ET



    
def sleep_extract_30s_epochs(path):
    try:
        root = ET.parse(path).getroot()
        events = [x for x in list(root.iter()) if x.tag == "ScoredEvent"]
        events_decomposed = list([list(event.iter()) for event in events])
        stage_events = [x for x in events_decomposed if x[1].text == "Stages|Stages"]
        starts = np.array([float(x[3].text) for x in stage_events])/30
        durations = np.array([float(x[4].text) for x in stage_events])/30
        stages = np.array([int(x[2].text[-1]) for x in stage_events])
        sleep_timeline: ndarray = np.zeros(int(starts[-1] + durations[-1]))
        for i in range(len(stages)):
            sleep_timeline[int(starts[i]): int(starts[i] + durations[i])] = stages[i]
        return sleep_timeline
    except:
        print(f"Could not extract sleep from: {path}")
        return 0

def duplicate(x):
    return np.array([[x_] * 2 for x_ in x]).flatten()

def hold_line(t, x):
    t = [i for i in t]
    x = [i for i in x]

    if len(t) - len(x) == 0:
        t.append(t[-1] + t[-1] - t[-2])
    if len(t) - len(x) != 1:
        raise ValueError()

    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_

def clean_sleep_2(sleep):
    for i in range(1, len(sleep)-1):
        if sleep[i] != sleep[i-1]:
            if sleep[i-1] == sleep[i+1]:
                sleep[i] = sleep[i-1]
    
    return sleep

def clean_sleep(sleep):
    new_sleep = copy.deepcopy(sleep)
    transitions = np.where(sleep[:-1] != sleep[1:])[0]
    print(transitions)
    for transition in transitions: 
        if sleep[transition-1]==sleep[transition+1]:
            new_sleep[transition] = sleep[transition-1]
    return new_sleep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion3/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser

plt.close('all')
dl = MESAParser()
plt.plot(dl.load_signal(4408, "PPG"))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
from scipy.signal import butter, resample, sosfiltfilt, cheby1

def _filter_signal(signal):
    sos = cheby1(8, 0.001, 8 / (256 / 2), 'lowpass', output='sos')
    return sosfiltfilt(sos, signal, axis=0)

In [3]:
def _resample_with_filter(signal):
    signal = _filter_signal(signal)
    signal = resample(signal, int(len(signal) / (256/32))).astype(np.float32)
    return signal

In [4]:
from multiprocessing import Pool

import tqdm

def check_valid(patient):
    signal = dl.load_signal(patient, 'Pleth')
    signal = _resample_with_filter(signal)

    if signal.std() == np.inf:
        print(patient, flush=True)

# with Pool(32) as p:
#     p.map(check_valid, dl.database_patient_IDs)
# Parallel(n_jobs=8)(delayed(check_valid)(patient) for patient in tqdm.tqdm(dl.database_patient_IDs))

In [5]:
dl = MESAParser()

patient = '6217'

signal = dl.load_signal(patient, 'Pleth')
ecg = dl.load_signal(patient, 'EKG')
signal = _resample_with_filter(signal)
sleep = dl.load_sleep(patient)

t_signal = np.arange(0, len(signal)/32, 1/32)
t_sleep = np.arange(0, len(sleep)*30, 30)
t_ecg = np.arange(0, len(ecg)/256, 1/256)

# plt.close('all')
plt.figure(figsize=(15,5))
ax1 = plt.subplot(313)
ax1.plot(t_signal, signal)
ax3 = plt.subplot(312, sharex=ax1)
ax3.plot(t_ecg, ecg)
ax2 = plt.subplot(311, sharex=ax1)
ax2.plot(t_sleep, sleep)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')
spect, freqs, t, im = plt.specgram(ihr, Fs=2, NFFT=128, noverlap=60)

In [ ]:
plt.close('all')
plt.figure(figsize=(14,8))
plt.imshow(np.log(spect), origin='lower', interpolation='nearest', aspect='auto')
plt.plot(sleep*10)

In [ ]:
import numpy as np

epoch_windows = 30
epoch_stride = 20


X = np.arange(10000)
X = X.reshape(10,100,10)

shape = (X.shape[0], int(int(X.shape[1] - epoch_windows)/epoch_stride+1), epoch_windows, X.shape[2])
strides = (X.strides[0], X.strides[1]*epoch_stride, X.strides[1], X.strides[2])
y = np.lib.stride_tricks.as_strided(X, shape=shape, strides=strides)
print(y.strides)
y = y.reshape(y.shape[0]*y.shape[1], y.shape[2], y.shape[3])
s = y.shape
print(y.strides)
z = y.flatten().reshape(s)
print(z.strides)

In [ ]:
import numpy as np
epoch_windows = 3
epoch_stride = 2

X = np.asarray(range(1,81), np.int64).reshape(2,8,5)
print('Shape: ', X.shape, 'Strides: ', X.strides)
print(X)
shape = (X.shape[0], int(int(X.shape[1] - epoch_windows)/epoch_stride+1), epoch_windows, X.shape[2])
strides = (X.strides[0], X.strides[1]*epoch_stride, X.strides[1], X.strides[2])
y = np.lib.stride_tricks.as_strided(X, shape=shape, strides=strides)
print(y.shape)
y[y>82] = -1
y[y<-1] = -1
print(y)

In [ ]:
x = np.asarray(range(1,21), np.int64).reshape(4,5)
print(x)
print(x.strides)
np.lib.stride_tricks.as_strided(x, shape=(3,1,5), strides=(40,40,8))


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal, interpolate
import copy
import os 
import pandas as pd
from tqdm import tqdm
from bishop_peaks_onsets import *
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion3/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
dl = MESAParser()
patients = dl.database_all_patient_IDs
patient = patients[100]
signal = dl.load_signal(patient, 'PPG')
qppg = dl.load_annotation(patient, "PPG",'qppg')
bishop = dl.load_annotation(patient, "PPG",'Bishop')


In [13]:
plt.close('all')
plt.plot(signal)
plt.plot(qppg, signal[qppg], 'x')
plt.plot(bishop, signal[bishop], 'o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …